In [ ]:
import pandas as pd
import numpy as np
import csv
import sklearn
import ast
import re
read_csv = open("/content/drive/MyDrive/recipe_data.csv", encoding='UTF8')
df = pd.read_csv(read_csv)
read_csv = open("/content/drive/MyDrive/token_list.csv", encoding='UTF8')
token_list = pd.read_csv(read_csv)
token_list = set(token_list['0'])

In [ ]:
import nltk
nltk.download('wordnet')
wnl = nltk.WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_df = 0.95, stop_words ='english')
cv_text = cv.fit_transform(df['tokens'])
features = cv.get_feature_names()

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 100)
lda.fit(cv_text)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=100, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [ ]:
#sentence = "low fat, and does not contain nuts"

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
#tag_predict = nltk.pos_tag(arr, tagset = "universal")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [ ]:
def query_process(sentence):
  arr = [wnl.lemmatize(word) for word in nltk.wordpunct_tokenize(sentence.lower())]
  tag_predict = nltk.pos_tag(arr, tagset = "universal")
  search_queue = []
  no_tag = set(['VERB', 'PRON', 'DET', '.', 'CONJ'])
  time_list = set(['h','m','hour','minute','min'])
  no_flag = False
  time_set = 1
  for element in tag_predict:
    if element[1] not in no_tag:
      if element[0] == 'no' or element[0] == 'not':
        no_flag = True
      elif element[1] == 'ADP':
        if element[0] == 'without':
          no_flag = True
      elif element[1] == 'NUM':
        time_set = int(element[0])
      elif element[0] in time_list:
        if element[0] == 'h' or element[0] == 'hour':
          search_queue.append(time_set*60)
        else:
          search_queue.append(time_set)
        continue
      else:
        if no_flag:
          search_queue.append("!"+element[0])
          no_flag = False
        else:
          search_queue.append(element[0])
  return search_queue

In [ ]:
def intents(word_list):
  #negative flag
  negative_word = []
  #nutrition
  nut_list = set(['sugars', 'fat', 'calories','sodium'])
  nut_word = []
  #time
  cook_time = []
  #나머지
  search_word = []

  for word in word_list:
    if isinstance(word, int):
      cook_time.append(word)
    elif word[0] == "!":
      negative_word.append(word[1:])
    elif word in nut_list:
      nut_word.append(word)
    else:
      if word in token_list:
        search_word.append(word)

  #topic1
  cv_query = cv.transform([' '.join(search_word)])
  query_topic = np.argmax(lda.transform(cv_query))

  return negative_word, nut_word, cook_time, search_word, query_topic

In [ ]:
# 쿼리 정리 : !오븐 피넛 토마토 라임 바질 10분 지방 -> 1. 오븐 2. 피넛 토마토 라임 바질 3. 10분 4. 지방 5. LDA 토픽
# 순서 : 5 bool filter -> 4 bool filter -> 3 bool filter -> 1 bool filter -> 2 score

In [ ]:
def search_recipe(query):
  word_list = query_process(query)
  negative_word, nut_word, cook_time, search_word, query_topic = intents(word_list)

  # Recipe_Name
  score_list = []
  for element in df['name']:
    res = ast.literal_eval(element)
    score = 0
    for word in search_word:
        if word in res:
          score += 1
    score_list.append(10*score*score/len(res))
  df['score'] = score_list

  # Topic
  topic_list = set(df.nlargest(10, 'score')['topic'].values.tolist())
  topic_list.add(query_topic)
  df_filter = pd.DataFrame(df.loc[df['topic'].isin(topic_list), :])

  # Nut
  for word in nut_word:
    df_filter = pd.DataFrame(df_filter.loc[df_filter[word], :])

  # Scoring
  score_list = df_filter['score'].values.tolist()

  # time
  tot_time = 0
  for time in cook_time:
    tot_time += time

  for idx, element in enumerate(df_filter['time']):
    time_element = int(element)
    if time_element >= tot_time:
      score_list[idx] -= (time_element-tot_time)/5

  # neg_filter + ingtool
  for idx, element in enumerate(df_filter['ing_tool']):
    ing_set = ast.literal_eval(element)
    score = 0
    for word in negative_word:
      if word in ing_set:
        score -= 4
    for word in search_word:
      if word in ing_set:
        score += 2
    score_list[idx] += score

  df_filter['score'] = score_list
  df_filter['score'] = df_filter.apply(lambda row: row['score'] + row['aver_rate']*(1 + 0.2*(min(row['review_nums']/100,1)))/5, axis = 1)
                                       
  
  return df_filter.nlargest(10, 'score')[['recipe_name', 'ingredients', 'time', 'recipe']]

In [ ]:
from tabulate import tabulate

In [ ]:
retry = True
search = True

while retry:
  retry = True
  search = True
  query = input("Recipe Search :")
  #query1 = "Give me a Korean food which could be made without oven. It should be low fat and take less than a hour."
  #query2 = "seafood pasta which contains tomato and basil and could be cooked less than 30 mins."
  result = search_recipe(query)
  while retry and search:
    search = True
    print(tabulate(result[['recipe_name']], headers='keys', tablefmt='psql', showindex=False))
    while True:
      try:
        click_recipe = int(input("Choose Recipe(1~10) :"))
        if click_recipe <= 10 and click_recipe >= 1:
          break
        else:
          print("Choose 1 ~ 10.")
          continue
      except ValueError:
        print("That was no valid number. Choose 1 ~ 10.")

    idx = result.index.values.tolist()[click_recipe-1]
    print("Ingredients : \n\n", result['ingredients'][idx].replace('^', '\n'), "\n")
    print("Cooking Time : ", result['time'][idx], " min\n")
    print("Cooking Direction : \n\n- ", result['recipe'][idx].replace('.', '.\n- '))

    while True:
      try:
        option = int(input("Back to list[1], Start new search[2], Stop[3] :"))
        if option <= 3 and option >= 1:
          break
        else:
          print("Choose 1,2 or 3.")
          continue
      except ValueError:
        print("Choose 1,2 or 3.")
    
    if option == 2:
      search = False
    if option == 3:
      retry = False

print("End")

Recipe Search :Give me a Korean food which could be made without oven. It should be low fat and take less than a hour.
+----------------------------------------------------------------+
| recipe_name                                                    |
|----------------------------------------------------------------|
| Korean Spicy Chicken and Potato (Tak Toritang)                 |
| Chompchae Deopbap (Korean Spicy Tuna and Rice)                 |
| Kimbop (Korean Sushi)                                          |
| Korean Marinade                                                |
| Fairy Food                                                     |
| Korean Squash                                                  |
| Korean Pizza                                                   |
| Quick and Simple Korean Doenjang Chigae (Bean Paste/Tofu Soup) |
| Vegan Japchae Korean Noodles                                   |
| Kwek Kwek (Filipino Street Food)                               |
+---------